In [ ]:
import numpy as np, torch, pickle, pandas as pd
from pathlib import Path
from typing import Union, List, Dict, Tuple, Optional, Callable
from dataclasses import dataclass
from fastcore.utils import *

def parse_portfolio(path):
    "Parse portfolio CSV and return cleaned equity and options dataframes"
    df = pd.read_csv(path, skiprows=1)
    df = df[~df['Symbol'].isin(['Cash & Cash Investments', 'Account Total'])].copy()
    equities = df[df['Security Type'].isin(['Equity', 'ETFs & Closed End Funds'])].copy()
    options = df[df['Security Type'] == 'Option'].copy()
    return equities, options


def clean_currency(val): return float(str(val).replace('$','').replace(',','').replace('--','0')) if pd.notna(val) else 0.0
def clean_pct(val): return float(str(val).replace('%','').replace('--','0'))/100 if pd.notna(val) else 0.0

def get_cash(path):
    "Extract cash balance from portfolio CSV"
    df = pd.read_csv(path, skiprows=1)
    cash_row = df[df['Symbol'] == 'Cash & Cash Investments']
    return clean_currency(cash_row.iloc[0]['Mkt Val (Market Value)']) if len(cash_row) > 0 else 0.0

def check_short_puts(options, equities, path):
    "Alert for short puts with insufficient cash coverage"
    alerts = []
    cash = get_cash(path)
    total_exposure = 0
    for _, opt in options.iterrows():
        if int(opt['Qty (Quantity)']) >= 0: continue
        o = parse_option_symbol(opt['Symbol'])
        if o['opt_type'] != 'P': continue
        qty = abs(int(opt['Qty (Quantity)']))
        exposure = qty * o['strike'] * 100
        total_exposure += exposure
    if total_exposure > cash:
        shortage = total_exposure - cash
        alerts.append(f"💰 Short puts require ${total_exposure:,.0f} cash but only ${cash:,.0f} available (${shortage:,.0f} short)")
    return alerts

def parse_option_symbol(sym):
    "Parse option symbol into components: (underlying, expiration, strike, type)"
    parts = sym.split()
    return dict(underlying=parts[0], exp=parts[1], strike=float(parts[2]), opt_type=parts[3])

def parse_portfolio(path):
    "Parse portfolio CSV and return cleaned equity and options dataframes"
    df = pd.read_csv(path, skiprows=1)
    df = df[~df['Symbol'].isin(['Cash & Cash Investments', 'Account Total'])].copy()
    equities = df[df['Security Type'].isin(['Equity', 'ETFs & Closed End Funds'])].copy()
    options = df[df['Security Type'] == 'Option'].copy()
    return equities, options

def check_expiring_options(options, equities, days=7):
    "Alert for options expiring within N days, with context-aware advice"
    alerts = []
    for _, opt in options.iterrows():
        o = parse_option_symbol(opt['Symbol'])
        exp_date = pd.to_datetime(o['exp']).normalize()
        today = pd.Timestamp.now().normalize()
        dte = (exp_date - today).days
        if dte > days: continue
        
        # Get underlying price if available
        eq = equities[equities['Symbol'] == o['underlying']]
        price = clean_currency(eq.iloc[0]['Price']) if len(eq) > 0 else None
        
        # Determine moneyness for short options
        qty = int(opt['Qty (Quantity)'])
        if qty < 0 and price:
            if o['opt_type'] == 'C': otm = price < o['strike'] * 0.95
            else: otm = price > o['strike'] * 1.05
            if otm:
                gain = clean_pct(opt['Gain % (Gain/Loss %)'])
                alerts.append(f"⏰ {o['underlying']}: {o['opt_type']} ${o['strike']} expires in {dte}d - deep OTM, let expire (+{gain*100:.0f}% profit)")
            else:
                alerts.append(f"⏰ {o['underlying']}: {o['opt_type']} ${o['strike']} expires in {dte}d - consider rolling or closing")
        else:
            alerts.append(f"⏰ {o['underlying']}: {o['opt_type']} ${o['strike']} expires in {dte}d")
    return alerts

def check_high_delta(options, threshold=0.5):
    "Alert for short options with high delta (assignment risk)"
    alerts = []
    for _, opt in options.iterrows():
        if int(opt['Qty (Quantity)']) >= 0: continue
        o = parse_option_symbol(opt['Symbol'])
        delta = abs(float(opt['Delta']))
        if delta > threshold: alerts.append(f"⚠️  {o['underlying']}: High Δ={delta:.2f} on short {o['opt_type']} ${o['strike']} - assignment risk")
    return alerts

def check_itm_options(options, equities):
    "Alert for short options that are in the money"
    alerts = []
    for _, opt in options.iterrows():
        if int(opt['Qty (Quantity)']) >= 0: continue
        o = parse_option_symbol(opt['Symbol'])
        eq = equities[equities['Symbol'] == o['underlying']]
        if len(eq) == 0: continue
        price = clean_currency(eq.iloc[0]['Price'])
        itm = (o['opt_type'] == 'C' and price > o['strike']) or (o['opt_type'] == 'P' and price < o['strike'])
        if itm: alerts.append(f"🚨 {o['underlying']}: Short {o['opt_type']} ${o['strike']} is ITM (price=${price:.2f})")
    return alerts

def check_unrealized_losses(equities, threshold=-0.10):
    "Alert for positions with large unrealized losses"
    alerts = []
    for _, eq in equities.iterrows():
        gain_pct = clean_pct(eq['Gain % (Gain/Loss %)'])
        if gain_pct < threshold: alerts.append(f"📉 {eq['Symbol']}: Down {gain_pct*100:.1f}% - review position")
    return alerts

def check_naked_options(options, equities):
    "Alert for short options without underlying shares"
    alerts = []
    for _, opt in options.iterrows():
        if int(opt['Qty (Quantity)']) >= 0: continue
        o = parse_option_symbol(opt['Symbol'])
        eq = equities[equities['Symbol'] == o['underlying']]
        if len(eq) == 0: alerts.append(f"⚠️  {o['underlying']}: Naked short {o['opt_type']} ${o['strike']} - no underlying held")
    return alerts


def portfolio_alerts(path):
    "Run all portfolio checks and return alerts"
    equities, options = parse_portfolio(path)
    alerts = []
    alerts += check_expiring_options(options, equities, days=7)
    alerts += check_high_delta(options, threshold=0.5)
    alerts += check_itm_options(options, equities)
    alerts += check_unrealized_losses(equities, threshold=-0.10)
    alerts += check_short_puts(options, equities, path)
    return alerts if alerts else ["✅ No immediate alerts"]


& `clean_currency` & `clean_pct` & `parse_option_symbol` & `parse_portfolio` & `check_expiring_options` & `check_high_delta` & `check_itm_options` & `check_unrealized_losses` & `check_short_puts` & `portfolio_alerts` & `parse_portfolio`

In [ ]:
!ls

 Individual-Positions-2025-12-15-131842.csv  'check portfolio.ipynb'
 Individual-Positions-2025-12-16-022640.csv   solveit_settings.json


In [ ]:
SHEET_PATH = "Individual-Positions-2025-12-16-022640.csv"

In [ ]:
equities, options = parse_portfolio(SHEET_PATH)

In [ ]:
equities

,Symbol,Description,Qty (Quantity),Price,Price Chng $ (Price Change $),Price Chng % (Price Change %),Mkt Val (Market Value),Day Chng $ (Day Change $),Day Chng % (Day Change %),Cost Basis,...,Gain % (Gain/Loss %),Reinvest?,Reinvest Capital Gains?,P/E Ratio (Price/Earnings Ratio),Ratings,Delta,Gamma,Theta,Security Type,Unnamed: 20
0,NVDA,NVIDIA CORP,100.0053,$175.77,$1.27,0.73%,"$17,578.34",$127.01,0.73%,"$20,401.00",...,-13.84%,Yes,NaN,43.32,B,NaN,NaN,NaN,Equity,NaN
1,PLTR,PALANTIR TECHNOLOGIES INCLASS A,100,$181.15,-$0.32,-0.18%,"$18,115.40",-$32.00,-0.18%,"$17,470.66",...,3.69%,No,NaN,425.20,C,NaN,NaN,NaN,Equity,NaN
2,SOFI,SOFI TECHNOLOGIES INC,500,$25.64,-$1.46,-5.39%,"$12,819.10",-$730.00,-5.39%,"$14,103.31",...,-9.11%,No,NaN,50.32,D,NaN,NaN,NaN,Equity,NaN
3,IBIT,ISHARES BITCOIN ETF,152,$48.68,-$2.54,-4.96%,"$7,399.50",-$386.08,-4.96%,"$9,966.64",...,-25.76%,No,NaN,NaN,--,NaN,NaN,NaN,ETFs & Closed End Funds,NaN


In [ ]:
options

,Symbol,Description,Qty (Quantity),Price,Price Chng $ (Price Change $),Price Chng % (Price Change %),Mkt Val (Market Value),Day Chng $ (Day Change $),Day Chng % (Day Change %),Cost Basis,...,Gain % (Gain/Loss %),Reinvest?,Reinvest Capital Gains?,P/E Ratio (Price/Earnings Ratio),Ratings,Delta,Gamma,Theta,Security Type,Unnamed: 20
4,AZN 01/16/2026 90.00 P,PUT ASTRAZENECA PLC $90 EXP 01/16/26,-3,$1.48,-$0.92,-38.41%,-$442.50,$275.97,38.41%,-$673.01,...,34.25%,NaN,NaN,NaN,-,-0.3648,0.0662,-0.0311,Option,NaN
5,IBIT 02/20/2026 65.00 C,CALL ISHR BITCOIN TR ETF$65 EXP 02/20/26,-1,$0.43,-$0.23,-34.92%,-$43.00,$23.07,34.92%,-$56.34,...,23.68%,NaN,NaN,NaN,-,0.0981,0.0171,-0.0130,Option,NaN
6,NVDA 01/23/2026 200.00 C,CALL NVIDIA CORP $200 EXP 01/23/26,-1,$1.52,-$0.37,-19.41%,-$151.50,$36.50,19.41%,-$189.34,...,19.99%,NaN,NaN,NaN,-,0.1513,0.0116,-0.0617,Option,NaN
7,PLTR 01/16/2026 175.00 C,CALL PALANTIR TECHNOLOGI$175 EXP 01/16/26,-1,$15.45,-$0.56,-3.49%,"-$1,545.00",$55.82,3.49%,-$974.34,...,-58.57%,NaN,NaN,NaN,-,0.6595,0.0142,-0.1556,Option,NaN
8,SOFI 12/19/2025 29.00 C,CALL SOFI TECHNOLOGIES I$29 EXP 12/19/25,-5,$0.07,-$0.17,-72.34%,-$32.50,$85.00,72.34%,-$581.69,...,94.41%,NaN,NaN,NaN,-,0.0743,0.0695,-0.0329,Option,NaN


In [ ]:
portfolio_alerts(SHEET_PATH)

['⏰ SOFI: C $29.0 expires in 3d - deep OTM, let expire (+94% profit)',
 '⚠️  PLTR: High Δ=0.66 on short C $175.0 - assignment risk',
 '🚨 PLTR: Short C $175.0 is ITM (price=$181.15)',
 '📉 NVDA: Down -13.8% - review position',
 '📉 IBIT: Down -25.8% - review position',
 '💰 Short puts require $27,000 cash but only $20,314 available ($6,686 short)']